In [2]:
pip install -U pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26822 sha256=998195da7066651193c54425717647bd669e29d39c72748a1dfc7dbe9792db33
  Stored in directory: c:\users\crie072\appdata\local\pip\cache\wheels\5c\4b\ec\41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

In [4]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200810


In [5]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [170]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202008_新規Vintage")

#input
new_cust = "NEW_CUST_Vintage.csv"
uri = "URI_Vintage.csv"

#out_put
result = "集計_新規Vintage"

print(new_cust)
print(uri)
print(result)

NEW_CUST_Vintage.csv
URI_Vintage.csv
集計_新規Vintage


新規Vintage CUSTデータの読込＆集計

In [179]:
df = pd.read_csv(data_dir /f"{new_cust}",encoding='shift-jis')
df.head()

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,no_of_new_account,cancel,no_of_account
0,20200331,2020,3,201803,ゴールド,オンライン,24,0,0,1,11
1,20200331,2020,3,201803,ゴールド,オンライン,24,1,0,0,9
2,20200331,2020,3,201803,ゴールド,店頭タブレット,24,0,0,1,136
3,20200331,2020,3,201803,ゴールド,店頭タブレット,24,1,0,0,47
4,20200331,2020,3,201803,ゴールド,郵送,24,0,0,6,128


In [180]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(1092, 11)
RangeIndex(start=0, stop=1092, step=1)
Index(['cp', 'year', 'month', 'kanyu_yymm', 'card', 'channel', 'mob', 'kzk_f',
       'no_of_new_account', 'cancel', 'no_of_account'],
      dtype='object')
cp                    int64
year                  int64
month                 int64
kanyu_yymm            int64
card                 object
channel              object
mob                   int64
kzk_f                 int64
no_of_new_account     int64
cancel                int64
no_of_account         int64
dtype: object


In [181]:
df['cp']=pd.to_datetime(df['cp'], format='%Y%m%d')
df['kanyu_yymmdd'] = df['kanyu_yymm']*100+1
df['kanyu_yymmdd'] = pd.to_datetime(df['kanyu_yymmdd'], format='%Y%m%d')

In [174]:
df.head()

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,no_of_new_account,ttt,no_of_account,kanyu_yymmdd
0,2020-03-31,2020,3,201803,ゴールド,オンライン,24,0,0,1,11,2018-03-01
1,2020-03-31,2020,3,201803,ゴールド,オンライン,24,1,0,0,9,2018-03-01
2,2020-03-31,2020,3,201803,ゴールド,店頭タブレット,24,0,0,1,136,2018-03-01
3,2020-03-31,2020,3,201803,ゴールド,店頭タブレット,24,1,0,0,47,2018-03-01
4,2020-03-31,2020,3,201803,ゴールド,郵送,24,0,0,6,128,2018-03-01


In [182]:
df['cp_date'] = df['cp'].dt.strftime('%Y/%m/%d')
df['kanyu_date'] = df['kanyu_yymmdd'].dt.strftime('%Y/%m/%d')

In [183]:
q = """
        SELECT
        kanyu_date
        ,card
        ,channel
        ,mob
        ,sum(no_of_new_account) as sum_new_account
        from 
        df
        where 
        mob  = 0
        group by
        kanyu_date,card,channel,mob
        """

df_new = sqldf(q, locals())
df_new.head(3)

,kanyu_date,card,channel,mob,sum_new_account
0,2020/03/01,ゴールド,オンライン,0,23
1,2020/03/01,ゴールド,店頭タブレット,0,88
2,2020/03/01,ゴールド,郵送,0,626


In [186]:
q = """
        SELECT
        kanyu_date
        ,card
        ,channel
        ,mob
        ,sum(no_of_account) as sum_account
        ,sum(cancel) as sum_kaiyaku
        from 
        df
        group by
        kanyu_date,card,channel,mob
        """

df_cust = sqldf(q, locals())
df_cust.head(3)

,kanyu_date,card,channel,mob,sum_account,sum_kaiyaku
0,2018/03/01,ゴールド,オンライン,24,20,1
1,2018/03/01,ゴールド,オンライン,25,17,3
2,2018/03/01,ゴールド,オンライン,26,17,0


In [191]:
q = """
        SELECT
        t1.kanyu_date
        ,t1.card
        ,t1.channel
        ,t1.mob
        ,t1.sum_account
        ,t1.sum_kaiyaku
        ,t2.sum_new_account
        from 
        df_cust t1
        left join 
        df_new t2
        on t1.kanyu_date = t2.kanyu_date
        and t1.card = t2.card
        and t1.channel = t2.channel
        order by 
        t1.kanyu_date,t1.card,t1.channel,t1.mob
        """

df_shukei = sqldf(q, locals())
df_shukei.head(3)

,kanyu_date,card,channel,mob,sum_account,sum_kaiyaku,sum_new_account
0,2018/03/01,ゴールド,オンライン,24,20,1,NaN
1,2018/03/01,ゴールド,オンライン,25,17,3,NaN
2,2018/03/01,ゴールド,オンライン,26,17,0,NaN


In [197]:
df_shukei['survive'] = df_shukei['sum_account']/df_shukei['sum_new_account']
df_shukei.head()

,kanyu_date,card,channel,mob,sum_account,sum_kaiyaku,sum_new_account,survive
0,2018/03/01,ゴールド,オンライン,24,20,1,NaN,NaN
1,2018/03/01,ゴールド,オンライン,25,17,3,NaN,NaN
2,2018/03/01,ゴールド,オンライン,26,17,0,NaN,NaN
3,2018/03/01,ゴールド,店頭タブレット,24,183,1,NaN,NaN
4,2018/03/01,ゴールド,店頭タブレット,25,176,7,NaN,NaN


In [198]:
df_shukei.to_csv(data_dir /'test.csv', header=True ,encoding='utf-8-sig')

In [ ]:
df_merge.to_csv(data_dir /r"output"/f"{result}.csv", header=True,encoding='utf-8-sig')

In [95]:
df['kanyu_yymmdd'] = pd.to_datetime(df['kanyu_yymmdd'], format = '%Y%m%d')
df['kanyu_year'] = df['kanyu_yymmdd'].dt.year
df['kanyu_month'] = df['kanyu_yymmdd'].dt.month
df['kanyu_yymm'] = df[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df.head()

,cp,no,kanyu_yymmdd,card,channel,kzk_f,kanyu_year,kanyu_month,kanyu_yymm
0,20180331,4980006342739,2018-03-01,ゴールド,店頭タブレット,1,2018,3,2018年3月
1,20180331,4980006342748,2018-03-01,ゴールド,店頭タブレット,0,2018,3,2018年3月
2,20180331,4980006342757,2018-03-01,ゴールド,店頭タブレット,0,2018,3,2018年3月
3,20180331,4980006342766,2018-03-01,ゴールド,郵送,1,2018,3,2018年3月
4,20180331,4980006342775,2018-03-01,ゴールド,郵送,1,2018,3,2018年3月


In [96]:
df['card'].unique()

array(['ゴールド', '一般', '外商'], dtype=object)

In [97]:
df['kanyu_yymm'].unique()

array(['2018年3月', '2018年4月', '2018年5月', '2018年6月', '2018年7月', '2018年8月',
       '2018年9月', '2018年10月', '2018年11月', '2018年12月', '2019年1月',
       '2019年2月', '2019年3月'], dtype=object)

In [98]:
#noの重複がないかチェック
print(len(df) == len(df['no'].unique()))

True


In [99]:
#重複削除
df_dup = df.drop_duplicates(subset=['no'])
df_dup.shape[0]

141347

In [100]:
#欠損データがないかチェック
df_dup.isnull().sum()

cp              0
no              0
kanyu_yymmdd    0
card            0
channel         0
kzk_f           0
kanyu_year      0
kanyu_month     0
kanyu_yymm      0
dtype: int64

集計

In [101]:
#客数
df_new= df_dup.groupby(['kanyu_yymm','card'])['no'].count()
df_new.head()

kanyu_yymm  card
2018年10月    ゴールド      462
            一般       9712
            外商       1214
2018年11月    ゴールド      485
            一般      11210
Name: no, dtype: int64

売上の集計

In [105]:
df = pd.read_csv(data_dir /r"input"/f"{uri_tot}",encoding='shift-jis')
df.head()

,kanyu_yymm,card,tot_amt_class,no_of_new_account
0,201803,ゴールド,21_100万円以上,36
1,201803,ゴールド,12_10万円以上,74
2,201803,ゴールド,3_1万円以上,28
3,201803,ゴールド,22_200万円以上,7
4,201803,ゴールド,13_20万円以上,63


In [106]:
#2019年3月までに限定する
df =df[(df["kanyu_yymm"] <=201903)]

In [107]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(977, 4)
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            967, 968, 969, 970, 971, 972, 973, 974, 975, 976],
           dtype='int64', length=977)
Index(['kanyu_yymm', 'card', 'tot_amt_class', 'no_of_new_account'], dtype='object')
kanyu_yymm            int64
card                 object
tot_amt_class        object
no_of_new_account     int64
dtype: object


In [108]:
df['kanyu_yymmdd']=df['kanyu_yymm']*100+1
df['kanyu_yymmdd'] = pd.to_datetime(df['kanyu_yymmdd'], format = '%Y%m%d')
df['kanyu_year'] = df['kanyu_yymmdd'].dt.year
df['kanyu_month'] = df['kanyu_yymmdd'].dt.month
df['kanyu_yymm'] = df[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df.head()

,kanyu_yymm,card,tot_amt_class,no_of_new_account,kanyu_yymmdd,kanyu_year,kanyu_month
0,2018年3月,ゴールド,21_100万円以上,36,2018-03-01,2018,3
1,2018年3月,ゴールド,12_10万円以上,74,2018-03-01,2018,3
2,2018年3月,ゴールド,3_1万円以上,28,2018-03-01,2018,3
3,2018年3月,ゴールド,22_200万円以上,7,2018-03-01,2018,3
4,2018年3月,ゴールド,13_20万円以上,63,2018-03-01,2018,3


In [109]:
df['card'].unique()

array(['ゴールド', '一般', '外商'], dtype=object)

In [110]:
df['kanyu_yymm'].unique()

array(['2018年3月', '2018年4月', '2018年5月', '2018年6月', '2018年7月', '2018年8月',
       '2018年9月', '2018年10月', '2018年11月', '2018年12月', '2019年1月',
       '2019年2月', '2019年3月'], dtype=object)

集計

In [113]:
#客数
df_uri = df.groupby(['kanyu_yymm','card','tot_amt_class'])['no_of_new_account'].sum()
df_uri.head()

kanyu_yymm  card  tot_amt_class
2018年10月    ゴールド  10_8万円以上          9
                  11_9万円以上          9
                  12_10万円以上        63
                  13_20万円以上        38
                  14_30万円以上        36
Name: no_of_new_account, dtype: int64

In [114]:
df_uri.head()

kanyu_yymm  card  tot_amt_class
2018年10月    ゴールド  10_8万円以上          9
                  11_9万円以上          9
                  12_10万円以上        63
                  13_20万円以上        38
                  14_30万円以上        36
Name: no_of_new_account, dtype: int64

Merge

In [126]:
df_uri.head()

kanyu_yymm  card  tot_amt_class
2018年10月    ゴールド  10_8万円以上          9
                  11_9万円以上          9
                  12_10万円以上        63
                  13_20万円以上        38
                  14_30万円以上        36
Name: no_of_new_account, dtype: int64

In [121]:
df_new.head()

kanyu_yymm  card
2018年10月    ゴールド      462
            一般       9712
            外商       1214
2018年11月    ゴールド      485
            一般      11210
Name: no, dtype: int64

In [144]:
df_merge = pd.merge(df_uri, df_new, left_on=['kanyu_yymm', 'card'], right_index=True)
df_merge.head()

no_of_new_account   no
kanyu_yymm card tot_amt_class                        
2018年10月   ゴールド 10_8万円以上                       9  462
                11_9万円以上                       9  462
                12_10万円以上                     63  462
                13_20万円以上                     38  462
                14_30万円以上                     36  462

In [141]:
df_merge = df_merge.rename(columns={'no_of_new_account':'no_of_account','no':'new_account'})
df_merge.head()

account  new_account
kanyu_yymm card tot_amt_class                      
2018年10月   ゴールド 10_8万円以上             9          462
                11_9万円以上             9          462
                12_10万円以上           63          462
                13_20万円以上           38          462
                14_30万円以上           36          462

In [143]:
df_merge.to_csv(data_dir /r"output"/f"{result}.csv", header=True,encoding='utf-8-sig')